**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

#### Execute the below cell to load the datasets

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import warnings
warnings.simplefilter('ignore')

In [2]:
#Loading data
books = pd.read_csv("books/books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [3]:
users = pd.read_csv('books/users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

In [4]:
ratings = pd.read_csv('books/ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

### Check no.of records and features given in each dataset

In [5]:
print("number of books", books.shape[0])

number of books 271360


In [6]:
print("number of users", users.shape[0])

number of users 278858


In [7]:
print("number of ratings", ratings.shape[0])

number of ratings 1149780


## Exploring books dataset

In [8]:
books.sample(2)

ISBN                        bookTitle         bookAuthor  \
41389  0060925132              Death of a Tall Man  Frances Lockridge   
90029  0374290202  Wild Meat and the Bully Burgers  Lois-Ann Yamanaka   

      yearOfPublication             publisher  \
41389              1994         Harpercollins   
90029              1996  Farrar Straus Giroux   

                                               imageUrlS  \
41389  http://images.amazon.com/images/P/0060925132.0...   
90029  http://images.amazon.com/images/P/0374290202.0...   

                                               imageUrlM  \
41389  http://images.amazon.com/images/P/0060925132.0...   
90029  http://images.amazon.com/images/P/0374290202.0...   

                                               imageUrlL  
41389  http://images.amazon.com/images/P/0060925132.0...  
90029  http://images.amazon.com/images/P/0374290202.0...

### Drop last three columns containing image URLs which will not be required for analysis

In [9]:
print("Number of columns before drop", books.shape[1])
books.drop(['imageUrlS', 'imageUrlM', 'imageUrlL'], axis=1, inplace=True)
print("Number of columns after drop", books.shape[1])
books.sample(2)

Number of columns before drop 8
Number of columns after drop 5


ISBN      bookTitle      bookAuthor yearOfPublication  \
187636  8440612710   Amor Profano  Katherine Dunn              1990   
61030   0586068872  The Doubleman      C. J. Koch              1986   

                                        publisher  
187636  Centro de Esportacion de Libros Espanoles  
61030                    HarperCollins Publishers

**yearOfPublication**

### Check unique values of yearOfPublication


In [10]:
print("Year of publication\n", books['yearOfPublication'].unique())

Year of publication
 [2002 2001 1991 1999 2000 1993 1996 1988 2004 1998 1994 2003 1997 1983
 1979 1995 1982 1985 1992 1986 1978 1980 1952 1987 1990 1981 1989 1984 0
 1968 1961 1958 1974 1976 1971 1977 1975 1965 1941 1970 1962 1973 1972
 1960 1966 1920 1956 1959 1953 1951 1942 1963 1964 1969 1954 1950 1967
 2005 1957 1940 1937 1955 1946 1936 1930 2011 1925 1948 1943 1947 1945
 1923 2020 1939 1926 1938 2030 1911 1904 1949 1932 1928 1929 1927 1931
 1914 2050 1934 1910 1933 1902 1924 1921 1900 2038 2026 1944 1917 1901
 2010 1908 1906 1935 1806 2021 '2000' '1995' '1999' '2004' '2003' '1990'
 '1994' '1986' '1989' '2002' '1981' '1993' '1983' '1982' '1976' '1991'
 '1977' '1998' '1992' '1996' '0' '1997' '2001' '1974' '1968' '1987' '1984'
 '1988' '1963' '1956' '1970' '1985' '1978' '1973' '1980' '1979' '1975'
 '1969' '1961' '1965' '1939' '1958' '1950' '1953' '1966' '1971' '1959'
 '1972' '1955' '1957' '1945' '1960' '1967' '1932' '1924' '1964' '2012'
 '1911' '1927' '1948' '1962' '2006' '1952' '1940

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [11]:
books[books['yearOfPublication']=='DK Publishing Inc']

ISBN                                          bookTitle  \
209538  078946697X  DK Readers: Creating the X-Men, How It All Beg...   
221678  0789466953  DK Readers: Creating the X-Men, How Comic Book...   

       bookAuthor  yearOfPublication  \
209538       2000  DK Publishing Inc   
221678       2000  DK Publishing Inc   

                                                publisher  
209538  http://images.amazon.com/images/P/078946697X.0...  
221678  http://images.amazon.com/images/P/0789466953.0...

In [12]:
books[books['yearOfPublication']=='Gallimard']

ISBN                                          bookTitle  \
220731  2070426769  Peuple du ciel, suivi de 'Les Bergers\";Jean-M...   

       bookAuthor yearOfPublication  \
220731       2003         Gallimard   

                                                publisher  
220731  http://images.amazon.com/images/P/2070426769.0...

### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

Totally 3 rows have to be dropped based on the check above

In [13]:
before_drop = books.shape[0]
books.drop(books[(books['yearOfPublication']=='DK Publishing Inc') | (books['yearOfPublication']=='Gallimard')].index, inplace=True)
print("Number of rows dropped", before_drop - books.shape[0])

Number of rows dropped 3


### Change the datatype of yearOfPublication to 'int'

In [14]:
books['yearOfPublication'].dtypes

dtype('O')

In [15]:
books['yearOfPublication'] = books['yearOfPublication'].apply(pd.to_numeric)
#verify if the dtypes is changed
books['yearOfPublication'].dtypes

dtype('int64')

### Drop NaNs in `'publisher'` column


In [16]:
print("NA values", books['publisher'].isna().sum())
print("Null values",  books['publisher'].isnull().sum())
books[books['publisher'].isna()]

NA values 2
Null values 2


ISBN        bookTitle       bookAuthor  yearOfPublication  \
128890  193169656X      Tyrant Moon  Elaine Corvidae               2002   
129037  1931696993  Finders Keepers  Linnea Sinclair               2001   

       publisher  
128890       NaN  
129037       NaN

2 rows have to be dropped based on the NANs

In [17]:
before_drop = books.shape[0]
books.drop(books[books['publisher'].isna()].index, inplace=True)
print("Number of rows dropped", before_drop - books.shape[0])

Number of rows dropped 2


## Exploring Users dataset

In [18]:
print(users.shape)
users.sample(2)

(278858, 3)


userID                        Location   Age
162092  162093  hampton, new brunswick, canada  20.0
117265  117266   midland park, new jersey, usa  42.0

### Get all unique values in ascending order for column `Age`

In [19]:
print("Number of unique values", users['Age'].nunique())

Number of unique values 165


In [20]:
print("Unique Age values\n", np.sort(users['Age'].unique()))

Unique Age values
 [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 113. 114. 115. 116. 118. 119. 123. 124. 127. 128. 132. 133. 136. 137.
 138. 140. 141. 143. 146. 147. 148. 151. 152. 156. 157. 159. 162. 168.
 172. 175. 183. 186. 189. 199. 200. 201. 204. 207. 208. 209. 210. 212.
 219. 220. 223. 226. 228. 229. 230. 231. 237. 239. 244.  nan]


Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

In [21]:
print("Number of rows to be replaced:", users[(users['Age']<5) | (users['Age']>90)].count()['Age'])

Number of rows to be replaced: 1312


In [22]:
users['Age'] = np.where((users['Age']<5) | (users['Age']>90), np.nan, users['Age'])
print("Checking if all rows are done:", (users[(users['Age']<5) | (users['Age']>90)].shape[0]) == 0)

Checking if all rows are done: True


### Replace null values in column `Age` with mean

In [24]:
users.Age = users.Age.fillna(users.Age.mean())
print("NA values after", users['Age'].isna().sum())

NA values after 0


### Change the datatype of `Age` to `int`

In [25]:
users['Age'].dtypes

dtype('float64')

In [26]:
users['Age'] = users['Age'].apply(np.int)

In [27]:
users['Age'].dtypes

dtype('int64')

In [29]:
print(np.sort(users.Age.unique()))

[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76
 77 78 79 80 81 82 83 84 85 86 87 88 89 90]


Only integers between 5 & 90 are present

## Exploring the Ratings Dataset

### check the shape

In [30]:
ratings.shape

(1149780, 3)

In [31]:
n_users = users.shape[0]
n_books = books.shape[0]

In [32]:
ratings.sample(2)

userID        ISBN  bookRating
235820   54898  0345384466           0
541144  130157  0671007718           0

### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [33]:
books.head(1)

ISBN            bookTitle          bookAuthor  yearOfPublication  \
0  0195153448  Classical Mythology  Mark P. O. Morford               2002   

                 publisher  
0  Oxford University Press

In [34]:
print("Number of rows ", ratings.shape[0], "; Rows to be dropped", ratings[~ratings['ISBN'].isin(books['ISBN'].unique())].shape[0])

Number of rows  1149780 ; Rows to be dropped 118650


In [35]:
ratings.drop(ratings[~ratings['ISBN'].isin(books['ISBN'].unique())].index, inplace=True)
print("Number of rows after drop", ratings.shape[0])

Number of rows after drop 1031130


### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [36]:
users.head(1)

userID            Location  Age
0       1  nyc, new york, usa   34

In [37]:
print("Number of rows ", ratings.shape[0], "; Rows to be dropped", ratings[~ratings['userID'].isin(users['userID'].unique())].shape[0])

Number of rows  1031130 ; Rows to be dropped 0


In [38]:
ratings.drop(ratings[~ratings['userID'].isin(users['userID'].unique())].index, inplace=True)
print("Number of rows after drop", ratings.shape[0])

Number of rows after drop 1031130


### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [39]:
ratings.head(1)

userID        ISBN  bookRating
0  276725  034545104X           0

In [40]:
print("Number of rows before drop", ratings.shape[0])
ratings.drop(ratings[ratings['bookRating']==0].index, inplace=True)
print("Number of rows after drop", ratings.shape[0])

Number of rows before drop 1031130
Number of rows after drop 383839


In [41]:
print(np.sort(ratings['bookRating'].unique()))

[ 1  2  3  4  5  6  7  8  9 10]


bookRating is now only between 1 to 10 is present now in the ratings dataset

### Find out which rating has been given highest number of times

In [42]:
print("Highest rating given is ", ratings.groupby(by='bookRating').count().sort_values(by='userID', ascending=False).head(1).index[0])

Highest rating given is  8


### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [43]:
ratings.head(2)

userID        ISBN  bookRating
1  276726  0155061224           5
3  276729  052165615X           3

In [44]:
users.head(2)

userID                   Location  Age
0       1         nyc, new york, usa   34
1       2  stockton, california, usa   18

In [45]:
users.shape

(278858, 3)

In [46]:
ratings.shape

(383839, 3)

In [47]:
print("Number of unique users before process", ratings['userID'].nunique())

Number of unique users before process 68091


In [49]:
#Get the count per user
user_rating_count = pd.DataFrame((ratings.groupby('userID').count().reset_index()[['userID', 'bookRating']]))
user_rating_count.columns=['userID', 'numRatings']
user_rating_count.sample()

userID  numRatings
7448   30926           1

In [50]:
user_rating_count.shape

(68091, 2)

In [51]:
#get users who have reviewerd more than 100
#filter the users data and ratings data for users lesser than the threshold rating count - 100
rated_atleast_count = 100
user_rating_count = user_rating_count[user_rating_count['numRatings'] >= rated_atleast_count]
print("Number of rows in ratings to be considered for further details", user_rating_count.numRatings.sum())

Number of rows in ratings to be considered for further details 103269


In [52]:
ratings.shape

(383839, 3)

In [53]:
ratings_filtered = ratings[ratings['userID'].isin(user_rating_count['userID'])]
ratings_filtered.head()

userID        ISBN  bookRating
1456  277427  002542730X          10
1458  277427  003008685X           8
1461  277427  0060006641          10
1465  277427  0060542128           7
1474  277427  0061009059           9

In [54]:
ratings_filtered.shape

(103269, 3)

### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [55]:
ratings_filtered.head(1)

userID        ISBN  bookRating
1456  277427  002542730X          10

In [56]:
#with userID as rows and ISBN as columns and average rating as values, table to be made with ratings as values
print("Table shape to be:", len(ratings_filtered['userID'].unique()), "X", len(ratings_filtered['ISBN'].unique()))

Table shape to be: 449 X 66572


In [57]:
#performing a pivot with userID as index and ISBN as columns will make the matrix of explicit ratings
#fill the nas with 0
ratings_pivot = ratings_filtered.pivot(index = 'userID', columns ='ISBN', values = 'bookRating').fillna(0)

In [58]:
print("Check the there are no nan's present", ratings_pivot.isnull().sum().sum())
ratings_pivot.head(5)

Check the there are no nan's present 0


ISBN    0000913154  0001046438  000104687X  0001047213  0001047973  \
userID                                                               
2033           0.0         0.0         0.0         0.0         0.0   
2110           0.0         0.0         0.0         0.0         0.0   
2276           0.0         0.0         0.0         0.0         0.0   
4017           0.0         0.0         0.0         0.0         0.0   
4385           0.0         0.0         0.0         0.0         0.0   

ISBN    000104799X  0001048082  0001053736  0001053744  0001055607  ...  \
userID                                                              ...   
2033           0.0         0.0         0.0         0.0         0.0  ...   
2110           0.0         0.0         0.0         0.0         0.0  ...   
2276           0.0         0.0         0.0         0.0         0.0  ...   
4017           0.0         0.0         0.0         0.0         0.0  ...   
4385           0.0         0.0         0.0         0.0         0.0  ...   

ISBN    B000092Q0A  B00009EF82  B00009NDAN  B0000DYXID  B0000T6KHI  \
userID                                                               
2033           0.0         0.0         0.0         0.0         0.0   
2110           0.0         0.0         0.0         0.0         0.0   
2276           0.0         0.0         0.0         0.0         0.0   
4017           0.0         0.0         0.0         0.0         0.0   
4385           0.0         0.0         0.0         0.0         0.0   

ISBN    B0000VZEJQ  B0000X8HIE  B00013AX9E  B0001I1KOG  B000234N3A  
userID                                                              
2033           0.0         0.0         0.0         0.0         0.0  
2110           0.0         0.0         0.0         0.0         0.0  
2276           0.0         0.0         0.0         0.0         0.0  
4017           0.0         0.0         0.0         0.0         0.0  
4385           0.0         0.0         0.0         0.0         0.0  

[5 rows x 66572 columns]

### Generate the predicted ratings using SVD with no.of singular values to be 50

In [59]:
#The Singular Value Decomposition is a highlight of linear algebra
#A = U . Sigma . V^T
#Where A is the real m x n matrix that we wish to decompose, 
#U is an m x n matrix, 
#Sigma is an m x n diagonal matrix
#V^T is the  transpose of an n x n matrix where T is a superscript

In [60]:
ratings_pivot.shape

(449, 66572)

In [61]:
# Singular-value decomposition
from numpy import array
from scipy.sparse.linalg import svds

In [62]:
U, sigma, Vt = svds(ratings_pivot, k = 50)

In [63]:
U.shape

(449, 50)

In [64]:
Vt.shape

(50, 66572)

In [65]:
sigma

array([147.92121613, 149.3438051 , 150.07400599, 152.20116297,
       152.87416391, 154.61308307, 154.80093432, 155.95760177,
       158.05646578, 159.21079484, 159.81670657, 162.01963916,
       162.77851768, 163.33054635, 166.02489324, 166.8162391 ,
       168.04972004, 170.77485167, 171.01325686, 173.29428498,
       174.57624968, 176.65724713, 178.61913749, 180.29517222,
       182.25079063, 184.10706957, 187.61687534, 189.75276623,
       190.96966388, 195.14643609, 199.83133018, 201.70083339,
       202.18713912, 203.48697581, 207.26449173, 209.92986988,
       213.23598777, 216.88280493, 224.26954726, 231.66186197,
       235.67095629, 249.9581775 , 252.02866425, 261.24756904,
       267.98197504, 281.0120779 , 293.69539562, 379.58327277,
       634.72875357, 680.30978318])

In [66]:
sigma = np.diag(sigma)

In [67]:
sigma.shape

(50, 50)

In [68]:
all_users_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [69]:
all_users_predicted_ratings.shape

(449, 66572)

In [70]:
preds_df = pd.DataFrame(all_users_predicted_ratings, columns = ratings_pivot.columns)

In [71]:
preds_df.head(2)

ISBN  0000913154  0001046438  000104687X  0001047213  0001047973  000104799X  \
0       0.025341   -0.002146   -0.001431   -0.002146   -0.002146    0.002971   
1      -0.010012   -0.003669   -0.002446   -0.003669   -0.003669    0.001075   

ISBN  0001048082  0001053736  0001053744  0001055607  ...  B000092Q0A  \
0       -0.00392    0.007035    0.007035    0.012316  ...    0.000180   
1        0.00144   -0.003500   -0.003500    0.001612  ...   -0.000363   

ISBN  B00009EF82  B00009NDAN  B0000DYXID  B0000T6KHI  B0000VZEJQ  B0000X8HIE  \
0       0.000226    0.042081   -0.016804   -0.080028    0.004746    0.028314   
1       0.000403    0.008142    0.001104   -0.029224    0.000999    0.002363   

ISBN  B00013AX9E  B0001I1KOG  B000234N3A  
0       0.000120   -0.001693    0.067503  
1      -0.000242    0.000029   -0.013059  

[2 rows x 66572 columns]

In [72]:
ratings_pivot.shape

(449, 66572)

In [73]:
preds_df.shape

(449, 66572)

In [74]:
#put back the userID in the pred_df dataframe for searching based on userID
preds_df['userID'] = ratings_pivot.index
preds_df.set_index('userID', inplace=True)

In [75]:
preds_df.head(2)

ISBN    0000913154  0001046438  000104687X  0001047213  0001047973  \
userID                                                               
2033      0.025341   -0.002146   -0.001431   -0.002146   -0.002146   
2110     -0.010012   -0.003669   -0.002446   -0.003669   -0.003669   

ISBN    000104799X  0001048082  0001053736  0001053744  0001055607  ...  \
userID                                                              ...   
2033      0.002971    -0.00392    0.007035    0.007035    0.012316  ...   
2110      0.001075     0.00144   -0.003500   -0.003500    0.001612  ...   

ISBN    B000092Q0A  B00009EF82  B00009NDAN  B0000DYXID  B0000T6KHI  \
userID                                                               
2033      0.000180    0.000226    0.042081   -0.016804   -0.080028   
2110     -0.000363    0.000403    0.008142    0.001104   -0.029224   

ISBN    B0000VZEJQ  B0000X8HIE  B00013AX9E  B0001I1KOG  B000234N3A  
userID                                                              
2033      0.004746    0.028314    0.000120   -0.001693    0.067503  
2110      0.000999    0.002363   -0.000242    0.000029   -0.013059  

[2 rows x 66572 columns]

### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [76]:
userID = 2110

In [77]:
user_id = 2 #2nd row in ratings matrix and predicted matrix

In [78]:
books.head(2)

ISBN            bookTitle            bookAuthor  yearOfPublication  \
0  0195153448  Classical Mythology    Mark P. O. Morford               2002   
1  0002005018         Clara Callan  Richard Bruce Wright               2001   

                 publisher  
0  Oxford University Press  
1    HarperFlamingo Canada

In [79]:
ratings.head(2)

userID        ISBN  bookRating
1  276726  0155061224           5
3  276729  052165615X           3

### Get the predicted ratings for userID `2110` and sort them in descending order

In [81]:
#From the predictions locate the userID and then sort it in descending to get the users predictions
sorted_user_predictions = preds_df.loc[userID].sort_values(ascending = False)
sorted_user_predictions = pd.DataFrame(sorted_user_predictions)
sorted_user_predictions

2110
ISBN                
059035342X  0.682444
0345370775  0.368946
0345384911  0.333624
043935806X  0.333209
044021145X  0.329336
0451151259  0.313295
0439139597  0.305088
0439064872  0.290587
0380759497  0.278563
0345353145  0.250941
0451167317  0.249254
0439136369  0.242676
0618002235  0.239957
0880389117  0.239552
0618002227  0.239242
0451160525  0.234959
0446310786  0.231819
0451173317  0.229402
0440213525  0.228038
0060392452  0.227935
0439136350  0.226968
0345335287  0.223613
1560768304  0.221496
0441845630  0.221496
0451156609  0.221396
0451180232  0.221054
0345317580  0.219552
0451142934  0.218949
0312980140  0.218348
0670835382  0.216858
...              ...
0553576925 -0.042664
0688088686 -0.043207
0786000899 -0.043301
0553567683 -0.043553
042518630X -0.043825
0671673661 -0.044025
0345361571 -0.044076
042517770X -0.044668
0446603090 -0.044738
0684195984 -0.047424
0684195976 -0.047690
0679405283 -0.047838
0553801430 -0.048401
0385316895 -0.048595
0385311109 -0.048595
0451403789 -0.048595
042518109X -0.048705
0399142851 -0.049460
0553285785 -0.050022
0553295977 -0.050099
0743206029 -0.052753
0399140093 -0.052800
0688142877 -0.053489
0425191281 -0.054608
0553564528 -0.055067
0671744577 -0.056397
0399141383 -0.057334
0553569783 -0.060981
0515127833 -0.061348
0553561618 -0.067188

[66572 rows x 1 columns]

### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [82]:
#1st get the ratings done by user 
user_rating_info = ratings_filtered[ratings_filtered['userID']==userID]
user_data = (user_rating_info.merge(users, how = 'left', left_on = 'userID', right_on = 'userID').sort_values(['bookRating'], ascending = False))

In [83]:
user_data

userID        ISBN  bookRating                        Location  Age
76     2110  067166865X          10  charlotte, north carolina, usa   34
52     2110  0590109715          10  charlotte, north carolina, usa   34
64     2110  0590629786          10  charlotte, north carolina, usa   34
63     2110  0590629778          10  charlotte, north carolina, usa   34
61     2110  059046678X          10  charlotte, north carolina, usa   34
55     2110  059035342X          10  charlotte, north carolina, usa   34
93     2110  0812505042          10  charlotte, north carolina, usa   34
54     2110  0590213040          10  charlotte, north carolina, usa   34
53     2110  0590109960          10  charlotte, north carolina, usa   34
82     2110  0679805265          10  charlotte, north carolina, usa   34
97     2110  097089726x          10  charlotte, north carolina, usa   34
83     2110  0679811761          10  charlotte, north carolina, usa   34
31     2110  0380570009          10  charlotte, north carolina, usa   34
46     2110  0486270718          10  charlotte, north carolina, usa   34
87     2110  0743486625          10  charlotte, north carolina, usa   34
36     2110  0394843509          10  charlotte, north carolina, usa   34
42     2110  0441000150          10  charlotte, north carolina, usa   34
77     2110  0671695304          10  charlotte, north carolina, usa   34
40     2110  0439240700          10  charlotte, north carolina, usa   34
65     2110  0590629794          10  charlotte, north carolina, usa   34
66     2110  0590629808          10  charlotte, north carolina, usa   34
10     2110  0345307674          10  charlotte, north carolina, usa   34
2      2110  0140022651          10  charlotte, north carolina, usa   34
78     2110  0671751174          10  charlotte, north carolina, usa   34
101    2110  1570420564          10  charlotte, north carolina, usa   34
6      2110  0345260627          10  charlotte, north carolina, usa   34
7      2110  0345283554          10  charlotte, north carolina, usa   34
9      2110  034528710X          10  charlotte, north carolina, usa   34
8      2110  0345283929          10  charlotte, north carolina, usa   34
11     2110  0345314255          10  charlotte, north carolina, usa   34
..      ...         ...         ...                             ...  ...
38     2110  0439176824           8  charlotte, north carolina, usa   34
99     2110  1561008931           7  charlotte, north carolina, usa   34
96     2110  093317490X           7  charlotte, north carolina, usa   34
88     2110  0786807555           7  charlotte, north carolina, usa   34
0      2110  0060987529           7  charlotte, north carolina, usa   34
74     2110  0671529714           7  charlotte, north carolina, usa   34
33     2110  0394553284           7  charlotte, north carolina, usa   34
18     2110  0373614896           7  charlotte, north carolina, usa   34
20     2110  0373619472           7  charlotte, north carolina, usa   34
21     2110  0373619499           7  charlotte, north carolina, usa   34
22     2110  0373619502           7  charlotte, north carolina, usa   34
69     2110  0590880780           7  charlotte, north carolina, usa   34
28     2110  037364292X           7  charlotte, north carolina, usa   34
32     2110  038513097x           7  charlotte, north carolina, usa   34
27     2110  0373642911           7  charlotte, north carolina, usa   34
34     2110  0394563131           7  charlotte, north carolina, usa   34
62     2110  059047054X           7  charlotte, north carolina, usa   34
41     2110  0440920221           6  charlotte, north carolina, usa   34
91     2110  0806512318           6  charlotte, north carolina, usa   34
26     2110  0373642857           6  charlotte, north carolina, usa   34
25     2110  0373642849           6  charlotte, north carolina, usa   34
68     2110  0590880748           6  charlotte, north carolina, usa   34
17     2110  0373614845           6  charlotte, north carolina,

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [84]:
books.head()

ISBN                                          bookTitle  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

             bookAuthor  yearOfPublication                   publisher  
0    Mark P. O. Morford               2002     Oxford University Press  
1  Richard Bruce Wright               2001       HarperFlamingo Canada  
2          Carlo D'Este               1991             HarperPerennial  
3      Gina Bari Kolata               1999        Farrar Straus Giroux  
4       E. J. W. Barber               1999  W. W. Norton &amp; Company

In [85]:
user_full_info = (user_data.merge(books, how = 'left', left_on = 'ISBN', right_on = 'ISBN').sort_values(['bookRating'], ascending = False))

In [86]:
user_full_info

userID        ISBN  bookRating                        Location  Age  \
0      2110  067166865X          10  charlotte, north carolina, usa   34   
19     2110  0590629794          10  charlotte, north carolina, usa   34   
21     2110  0345307674          10  charlotte, north carolina, usa   34   
22     2110  0140022651          10  charlotte, north carolina, usa   34   
23     2110  0671751174          10  charlotte, north carolina, usa   34   
24     2110  1570420564          10  charlotte, north carolina, usa   34   
25     2110  0345260627          10  charlotte, north carolina, usa   34   
26     2110  0345283554          10  charlotte, north carolina, usa   34   
27     2110  034528710X          10  charlotte, north carolina, usa   34   
28     2110  0345283929          10  charlotte, north carolina, usa   34   
29     2110  0345314255          10  charlotte, north carolina, usa   34   
31     2110  0345335287          10  charlotte, north carolina, usa   34   
32     2110  0345362276          10  charlotte, north carolina, usa   34   
33     2110  0345375580          10  charlotte, north carolina, usa   34   
34     2110  1565111575          10  charlotte, north carolina, usa   34   
35     2110  0439222303          10  charlotte, north carolina, usa   34   
1      2110  0590109715          10  charlotte, north carolina, usa   34   
20     2110  0590629808          10  charlotte, north carolina, usa   34   
30     2110  0345317580          10  charlotte, north carolina, usa   34   
18     2110  0439240700          10  charlotte, north carolina, usa   34   
9      2110  0679805265          10  charlotte, north carolina, usa   34   
17     2110  0671695304          10  charlotte, north carolina, usa   34   
3      2110  0590629778          10  charlotte, north carolina, usa   34   
4      2110  059046678X          10  charlotte, north carolina, usa   34   
5      2110  059035342X          10  charlotte, north carolina, usa   34   
6      2110  0812505042          10  charlotte, north carolina, usa   34   
7      2110  0590213040          10  charlotte, north carolina, usa   34   
8      2110  0590109960          10  charlotte, north carolina, usa   34   
2      2110  0590629786          10  charlotte, north carolina, usa   34   
10     2110  097089726x          10  charlotte, north carolina, usa   34   
..      ...         ...         ...                             ...  ...   
57     2110  0671532642           8  charlotte, north carolina, usa   34   
89     2110  059047054X           7  charlotte, north carolina, usa   34   
88     2110  0394563131           7  charlotte, north carolina, usa   34   
87     2110  0373642911           7  charlotte, north carolina, usa   34   
86     2110  038513097x           7  charlotte, north carolina, usa   34   
85     2110  037364292X           7  charlotte, north carolina, usa   34   
84     2110  0590880780           7  charlotte, north carolina, usa   34   
83     2110  0373619502           7  charlotte, north carolina, usa   34   
82     2110  0373619499           7  charlotte, north carolina, usa   34   
81     2110  0373619472           7  charlotte, north carolina, usa   34   
79     2110  0394553284           7  charlotte, north carolina, usa   34   
78     2110  0671529714           7  charlotte, north carolina, usa   34   
77     2110  0060987529           7  charlotte, north carolina, usa   34   
76     2110  0786807555           7  charlotte, north carolina, usa   34   
75     2110  093317490X           7  charlotte, north carolina, usa   34   
74     2110  1561008931           7  charlotte, north carolina, usa   34   
80     2110  0373614896           7  charlotte, north carolina, usa   34   
94     2110  0590880748           6  charlotte, north carolina, usa   34   
97     2110  1572970944           6  charlotte, north carolina, usa   34   
96     2110  0345423038           6  charlotte, north carolina, usa   34   
95     2110  0373614845           6  charlotte, north

### Get top 10 recommendations for above given userID from the books not already rated by that user

In [87]:
user_rating_info.head(1)

userID        ISBN  bookRating
14448    2110  0060987529           7

In [88]:
sorted_user_predictions.shape

(66572, 1)

In [89]:
#the sorted_user_predictions has the users prediction order. 
#Remove the list of already predicted ISBN's which we have in the user_rating_info
#Take the head since it is already sorted
recommendations = sorted_user_predictions[~sorted_user_predictions.index.isin(np.array(user_rating_info['ISBN']))].head(10)
recommendations

2110
ISBN                
0345370775  0.368946
0345384911  0.333624
043935806X  0.333209
044021145X  0.329336
0451151259  0.313295
0439139597  0.305088
0439064872  0.290587
0380759497  0.278563
0345353145  0.250941
0451167317  0.249254

In [91]:
#do a merge to get the book information with the top 10 recommendations
print("List of top 10 recommendations which the user has not reviewed")
recommendations.merge(books, how='inner', on='ISBN')

List of top 10 recommendations which the user has not reviewed


ISBN      2110                                          bookTitle  \
0  0345370775  0.368946                                      Jurassic Park   
1  0345384911  0.333624                                       Crystal Line   
2  043935806X  0.333209  Harry Potter and the Order of the Phoenix (Boo...   
3  044021145X  0.329336                                           The Firm   
4  0451151259  0.313295                                 Eyes of the Dragon   
5  0439139597  0.305088       Harry Potter and the Goblet of Fire (Book 4)   
6  0439064872  0.290587   Harry Potter and the Chamber of Secrets (Book 2)   
7  0380759497  0.278563                 Xanth 15: The Color of Her Panties   
8  0345353145  0.250941                                             Sphere   
9  0451167317  0.249254                                      The Dark Half   

         bookAuthor  yearOfPublication                     publisher  
0  Michael Crichton               1999              Ballantine Books  
1    Anne McCaffrey               1993                 Del Rey Books  
2     J. K. Rowling               2003                    Scholastic  
3      John Grisham               1992  Bantam Dell Publishing Group  
4      Stephen King               1988           Penguin Putnam~mass  
5     J. K. Rowling               2000                    Scholastic  
6     J. K. Rowling               2000                    Scholastic  
7     Piers Anthony               1992                           Eos  
8  MICHAEL CRICHTON               1988              Ballantine Books  
9      Stephen King               1994                   Signet Book